# name-shivansh naui

# Data_preprocessing

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

data = pd.read_csv('IMDB Dataset.csv')

# Initialize NLTK resources
#it should always be uptodated 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Preprocessing function

In [2]:

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'http\S+', '', text)
    words = word_tokenize(text)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words and word.isalnum()]
    return ' '.join(filtered_words)



In [3]:
# Apply preprocessing to each review
data['cleaned_review'] = data['review'].apply(preprocess_text)

# Replace original review column with cleaned_review
data['review'] = data['cleaned_review']

# Drop the cleaned_review column if you don't need it anymore
data.drop(columns=['cleaned_review'], inplace=True)

# Save the cleaned data back to the same file
data.to_csv('Cleaned_Data.csv', index=False)

#  Vectorize Data Using:
    

# a) CountVectorizer


In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd


data = pd.read_csv('Cleaned_Data.csv')

# Initialize the CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the text data
count_vectors = count_vectorizer.fit_transform(data['review'])

# Store the sparse matrix directly in the DataFrame
data['count_vectors'] = list(count_vectors)

# Save the data with count vectors to a new CSV file
data.to_csv('data_with_count_vectors.csv', index=False)

#    b) TFIDFVectorizer
   

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

data = pd.read_csv('Cleaned_Data.csv')

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
tfidf_vectors = tfidf_vectorizer.fit_transform(data['review'])

# Store the sparse matrix directly in the DataFrame
data['tfidf_vectors'] = list(tfidf_vectors)

# Save the data with TF-IDF vectors to a new CSV file
data.to_csv('data_with_tfidf_vectors.csv', index=False)


#  c) Word2Vec

In [4]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np

# Load your review dataset
data = pd.read_csv("Cleaned_Data.csv")  # Replace "your_dataset.csv" with the actual file name


reviews = data['review']




In [5]:
# Tokenize the reviews into lists of words
tokenized_reviews = [review.split() for review in reviews]

# Train a Word2Vec model on your tokenized data
model = Word2Vec(sentences=tokenized_reviews, vector_size=300, window=5, min_count=1, workers=4)

# Function to convert a review text to a vector using the trained Word2Vec model
def vectorize_review(review):
    words = review.split()
    vectors = []
    for word in words:
        if word in model.wv:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)  # Average of word vectors
    else:
        return np.zeros(model.vector_size)  # Return zeros if no valid words are found



In [6]:
# Vectorize each review in the dataset
review_vectors = [vectorize_review(review) for review in reviews]

# Create a DataFrame to hold the review vectors
vector_df = pd.DataFrame(review_vectors)

# Concatenate the vector DataFrame with the original dataset
vectorized_data = pd.concat([data, vector_df], axis=1)

# Save the vectorized data to a new CSV file
vectorized_data.to_csv("vectorized_reviews_custom_word2vec.csv", index=False)

#     d) GoogleNews Word2Vec

In [6]:
import gensim.downloader as api

# Download the Google News Word2Vec model
model = api.load("word2vec-google-news-300")

# Save the model to a file
model.save("google_news_word2vec.model")



In [3]:
import pandas as pd
from gensim.models import KeyedVectors

# Load the saved KeyedVectors model
model = KeyedVectors.load("google_news_word2vec.model")

# Load your review dataset
data = pd.read_csv("Cleaned_Data.csv") 
reviews = data['review']

# Function to convert a review text to a vector using the KeyedVectors model
def vectorize_review(review):
    words = review.split()
    vector = []
    for word in words:
        if word in model:
            vector.append(model[word])
    if vector:
        return sum(vector) / len(vector)  # Average of word vectors
    else:
        return []

# Vectorize each review in the dataset
review_vectors = [vectorize_review(review) for review in reviews]

# Create a DataFrame to hold the review vectors
vector_df = pd.DataFrame(review_vectors)

# Concatenate the vector DataFrame with the original dataset
vectorized_data = pd.concat([data, vector_df], axis=1)

# Save the vectorized data to a new CSV file
vectorized_data.to_csv("vectorized_reviews.csv", index=False)


# Use following ML algorithms

# a) Logistic Regression
# CountVectorizer

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("data_with_count_vectors.csv")

# Extract the review text, sentiment, and count_vectors
reviews = data['review']
sentiments = data['sentiment']

# Use CountVectorizer to convert text data to numerical features
vectorizer = CountVectorizer()
count_vectors = vectorizer.fit_transform(reviews)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(count_vectors, sentiments, test_size=0.2, random_state=42)

# Create a Logistic Regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
report = classification_report(y_test, y_pred)
print(report)

C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8816
              precision    recall  f1-score   support

    negative       0.88      0.88      0.88      4961
    positive       0.88      0.89      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



#  TFIDFVectorizer

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix
import re

data = pd.read_csv('data_with_tfidf_vectors.csv')

# Function to convert the string representation to a sparse matrix
def parse_sparse_string(s):
    data = re.findall(r"\((\d+), ([\d.]+)\)", s)
    indices = [int(idx) for idx, _ in data]
    values = [float(val) for _, val in data]
    return csr_matrix((values, ([0] * len(indices), indices)), shape=(1, max(indices) + 1))

# Convert the string TF-IDF vectors to sparse matrices
X_sparse = data['tfidf_vectors'].apply(parse_sparse_string)

# Convert sparse matrices to dense arrays
X = [x.toarray().flatten() for x in X_sparse]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.4961
              precision    recall  f1-score   support

    negative       0.50      1.00      0.66      4961
    positive       0.00      0.00      0.00      5039

    accuracy                           0.50     10000
   macro avg       0.25      0.50      0.33     10000
weighted avg       0.25      0.50      0.33     10000



C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Word2Vec

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the dataset into a DataFrame
data = pd.read_csv('vectorized_reviews_custom_word2vec.csv')

# Separate features and target variable
X = data.drop(['review', 'sentiment'], axis=1)
y = data['sentiment']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict sentiments on the testing data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


C:\Users\ASUS\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.8695
              precision    recall  f1-score   support

    negative       0.87      0.86      0.87      4961
    positive       0.86      0.88      0.87      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



# GoogleNews 

In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Load the data (replace 'data.csv' with your actual dataset file)
data = pd.read_csv('data_with_google_news_word2vec..csv')

# Separate features (vectors) and labels
X = data.drop(['review', 'sentiment'], axis=1)  # Features are columns after 'review' and 'sentiment'
y = data['sentiment']  # Sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8497
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      4961
    positive       0.85      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000




#  SVC with CountVectorizer

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("data_with_count_vectors.csv")
data=data.head(10000)
# Extract the review text, sentiment, and count_vectors
reviews = data['review']
sentiments = data['sentiment']

# Use CountVectorizer to convert text data to numerical features
vectorizer = CountVectorizer()
count_vectors = vectorizer.fit_transform(reviews)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(count_vectors, sentiments, test_size=0.2, random_state=42)

# Create an SVC model
model = SVC()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.852
              precision    recall  f1-score   support

    negative       0.88      0.81      0.85       996
    positive       0.83      0.89      0.86      1004

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



# SVC with TFIDFVectorizer

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix
import re

data = pd.read_csv('data_with_tfidf_vectors.csv')

# Function to convert the string representation to a sparse matrix
def parse_sparse_string(s):
    data = re.findall(r"\((\d+), ([\d.]+)\)", s)
    indices = [int(idx) for idx, _ in data]
    values = [float(val) for _, val in data]
    return csr_matrix((values, ([0] * len(indices), indices)), shape=(1, max(indices) + 1))

# Convert the string TF-IDF vectors to sparse matrices
X_sparse = data['tfidf_vectors'].apply(parse_sparse_string)

# Convert sparse matrices to dense arrays
X = [x.toarray().flatten() for x in X_sparse]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize the Support Vector Classification model
model = SVC()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.5134
              precision    recall  f1-score   support

    negative       0.51      0.86      0.64      4961
    positive       0.55      0.18      0.27      5039

    accuracy                           0.51     10000
   macro avg       0.53      0.52      0.45     10000
weighted avg       0.53      0.51      0.45     10000



# SVC with Word2Vec

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset into a DataFrame
data = pd.read_csv('vectorized_reviews_custom_word2vec.csv')

# Separate features and target variable
X = data.drop(['review', 'sentiment'], axis=1)
y = data['sentiment']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Support Vector Classifier (SVC) model
model = SVC()
model.fit(X_train, y_train)

# Predict sentiments on the testing data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8733
              precision    recall  f1-score   support

    negative       0.88      0.86      0.87      4961
    positive       0.87      0.89      0.88      5039

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



# SVC Google new

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Load the data (replace 'data.csv' with your actual dataset file)
data = pd.read_csv('data_with_google_news_word2vec..csv')

# Separate features (vectors) and labels
X = data.drop(['review', 'sentiment'], axis=1)  # Features are columns after 'review' and 'sentiment'
y = data['sentiment']  # Sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Support Vector Classifier (SVC) model
model = SVC()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8599
              precision    recall  f1-score   support

    negative       0.86      0.85      0.86      4961
    positive       0.86      0.87      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



# RF with CountVectorizer

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv("data_with_count_vectors.csv")
data = data.head(10000)

# Extract the review text, sentiment, and count_vectors
reviews = data['review']
sentiments = data['sentiment']

# Use CountVectorizer to convert text data to numerical features
vectorizer = CountVectorizer()
count_vectors = vectorizer.fit_transform(reviews)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(count_vectors, sentiments, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier model
model = RandomForestClassifier()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8515
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85       996
    positive       0.85      0.86      0.85      1004

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



# RF TFIDFVectorizer

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from scipy.sparse import csr_matrix
import re

data = pd.read_csv('data_with_tfidf_vectors.csv')

# Function to convert the string representation to a sparse matrix
def parse_sparse_string(s):
    data = re.findall(r"\((\d+), ([\d.]+)\)", s)
    indices = [int(idx) for idx, _ in data]
    values = [float(val) for _, val in data]
    return csr_matrix((values, ([0] * len(indices), indices)), shape=(1, max(indices) + 1))

# Convert the string TF-IDF vectors to sparse matrices
X_sparse = data['tfidf_vectors'].apply(parse_sparse_string)

# Convert sparse matrices to dense arrays
X = [x.toarray().flatten() for x in X_sparse]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, data['sentiment'], test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.517
              precision    recall  f1-score   support

    negative       0.51      0.53      0.52      4961
    positive       0.52      0.51      0.52      5039

    accuracy                           0.52     10000
   macro avg       0.52      0.52      0.52     10000
weighted avg       0.52      0.52      0.52     10000



 #   Word2Vec

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset into a DataFrame
data = pd.read_csv('vectorized_reviews_custom_word2vec.csv')

# Separate features and target variable
X = data.drop(['review', 'sentiment'], axis=1)
y = data['sentiment']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Classifier model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Predict sentiments on the testing data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8401
              precision    recall  f1-score   support

    negative       0.85      0.82      0.84      4961
    positive       0.83      0.86      0.84      5039

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



#   GoogleNews Word2Vec 

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load the data (replace 'data.csv' with your actual dataset file)
data = pd.read_csv('data_with_google_news_word2vec..csv')

# Separate features (vectors) and labels
X = data.drop(['review', 'sentiment'], axis=1)  # Features are columns after 'review' and 'sentiment'
y = data['sentiment']  # Sentiment labels

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Random Forest Classifier model
model = RandomForestClassifier()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Evaluate the model's performance
report = classification_report(y_test, y_pred)
print(report)


Accuracy: 0.8102
              precision    recall  f1-score   support

    negative       0.81      0.80      0.81      4961
    positive       0.81      0.82      0.81      5039

    accuracy                           0.81     10000
   macro avg       0.81      0.81      0.81     10000
weighted avg       0.81      0.81      0.81     10000

